# From Scratch

In [1]:
import numpy as np
import cv2
import os
import time
import matplotlib.pyplot as plt

def read_images_from_folder(folder, resize=None):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), 0)
        if img is not None:
            if resize is not None:
                img = cv2.resize(img, resize)
            images.append(img)
    return images

def sobel_x(image):
    # Define Sobel kernel for horizontal derivative
    sobel_kernel_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    rows, cols = image.shape
    sobel_x_img = np.zeros_like(image, dtype=np.float64)
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            sobel_x_img[i, j] = np.sum(image[i-1:i+2, j-1:j+2] * sobel_kernel_x)

    return sobel_x_img

def sobel_y(image):
    sobel_kernel_y = np.array([[-1, -2, -1],
                               [0, 0, 0],
                               [1, 2, 1]])

    rows, cols = image.shape
    sobel_y_img = np.zeros_like(image, dtype=np.float64)
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            sobel_y_img[i, j] = np.sum(image[i-1:i+2, j-1:j+2] * sobel_kernel_y)

    return sobel_y_img

In [2]:
def apply_kernel(image, kernel):
    rows, cols = image.shape
    result = np.zeros_like(image, dtype=np.float64)
    pad_width = kernel.shape[0] // 2
    padded_image = np.pad(image, pad_width, mode='constant', constant_values=0)
    
    for i in range(pad_width, rows + pad_width):
        for j in range(pad_width, cols + pad_width):
            region = padded_image[i-pad_width:i+pad_width+1, j-pad_width:j+pad_width+1]
            result[i-pad_width, j-pad_width] = np.sum(region * kernel)
    
    return result

def gaussian_kernel(size, sigma):
    ax = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma))
    return kernel / np.sum(kernel)

def gaussian_blur(image, kernel_size=5, sigma=1):
    kernel = gaussian_kernel(kernel_size, sigma)
    return apply_kernel(image, kernel)

In [3]:
def harris_corner_detection(image, window_size=3, k=0.04, threshold=0.01):
    # Sobel operators for edge detection
    sobelX = sobel_x(image)
    sobelY = sobel_y(image)

    # Compute elements of the Harris matrix H
    Ixx = sobelX**2
    Iyy = sobelY**2
    Ixy = sobelX * sobelY

    # Gaussian smoothing
    Sxx = gaussian_blur(Ixx, window_size, 1.5)
    Syy = gaussian_blur(Iyy, window_size, 1.5)
    Sxy = gaussian_blur(Ixy, window_size, 1.5)

    # Compute the determinant and trace of the Harris matrix
    det_H = Sxx * Syy - Sxy**2
    trace_H = Sxx + Syy

    # Compute the Harris response
    harris_response = det_H - k * (trace_H**2)

    # Thresholding
    corners = np.zeros_like(image)
    corners[harris_response > threshold * harris_response.max()] = 255

    return corners

# mark corners with color
def color_corners(image, corners):
    colored_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    colored_image[corners > 0] = [0, 0, 255]  
    return colored_image

In [4]:
folder_path = "images_folder"
output_folder = "output_corners"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

images = read_images_from_folder(folder_path, resize=(512, 512))

window_size = 3  
k = 0.04 
threshold = 0.01  

for idx, img in enumerate(images):
    corners = harris_corner_detection(img, window_size, k, threshold)
    colored_corners = color_corners(img, corners)
    output_path = os.path.join(output_folder, f"corners_{idx}.jpg")
    cv2.imwrite(output_path, colored_corners)

# OpenCV approach and Comparision

In [5]:
# OpenCV Corner Detection
def corner_detection_opencv(image, max_corners=1000, quality_level=0.01, min_distance=10):
    gray = np.float32(image)
    corners = cv2.goodFeaturesToTrack(gray, maxCorners=max_corners, qualityLevel=quality_level, minDistance=min_distance)
    corners = np.intp(corners) if corners is not None else np.array([])
    return corners

def evaluate_performance(image):
    # Convert image to float32 for consistency in both methods
    image = np.float32(image)

    # Measure execution time for custom algorithm
    start_time_custom = time.time()
    corners_custom = harris_corner_detection(image)  # Ensure your function is defined to handle float32 images
    end_time_custom = time.time()

    # Measure execution time for OpenCV corner detection
    start_time_opencv = time.time()
    corners_opencv = corner_detection_opencv(image)
    end_time_opencv = time.time()

    # Calculate execution times
    execution_time_custom = end_time_custom - start_time_custom
    execution_time_opencv = end_time_opencv - start_time_opencv

    # Calculate accuracy (e.g., number of detected corners)
    accuracy_custom = np.sum(corners_custom > 0)
    accuracy_opencv = len(corners_opencv)

    return execution_time_custom, execution_time_opencv, accuracy_custom, accuracy_opencv

# Load benchmark images
# images = set of images
benchmark_results = []

for idx, image in enumerate(images):
    exec_time_custom, exec_time_opencv, acc_custom, acc_opencv = evaluate_performance(image)
    benchmark_results.append((exec_time_custom, exec_time_opencv, acc_custom, acc_opencv))

# Print results
for idx, result in enumerate(benchmark_results):
    print(f"Image {idx+1}:")
    print(f"Custom Algorithm Execution Time: {result[0]} seconds, Accuracy: {result[2]} corners")
    print(f"OpenCV Execution Time: {result[1]} seconds, Accuracy: {result[3]} corners")
    print()

Image 1:
Custom Algorithm Execution Time: 19.600157976150513 seconds, Accuracy: 275 corners
OpenCV Execution Time: 0.012239456176757812 seconds, Accuracy: 10 corners

Image 2:
Custom Algorithm Execution Time: 19.700519561767578 seconds, Accuracy: 10923 corners
OpenCV Execution Time: 0.011157751083374023 seconds, Accuracy: 206 corners

Image 3:
Custom Algorithm Execution Time: 19.577415943145752 seconds, Accuracy: 341 corners
OpenCV Execution Time: 0.012098550796508789 seconds, Accuracy: 19 corners

Image 4:
Custom Algorithm Execution Time: 19.03649592399597 seconds, Accuracy: 5848 corners
OpenCV Execution Time: 0.01168203353881836 seconds, Accuracy: 323 corners

Image 5:
Custom Algorithm Execution Time: 19.546329975128174 seconds, Accuracy: 17355 corners
OpenCV Execution Time: 0.015398263931274414 seconds, Accuracy: 1000 corners

Image 6:
Custom Algorithm Execution Time: 18.75178360939026 seconds, Accuracy: 925 corners
OpenCV Execution Time: 0.011022567749023438 seconds, Accuracy: 231 

In [6]:
output_folder = "output_openCV"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Define the color_corners function
def color_corners_openCV(image, corners):
    colored_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(colored_image, (x, y), 3, (0, 0, 255), -1)  # Red circles
    return colored_image

for idx, img in enumerate(images):
    corners = corner_detection_opencv(img)
    colored_corners = color_corners_openCV(img, corners)
    output_path = os.path.join(output_folder, f"corners_{idx}.jpg")
    cv2.imwrite(output_path, colored_corners)